# Doric

<<<<<<< local
<img src="images/dorict.png" alt="drawing" width="200"/>
=======
<img src="dorict.png" alt="drawing" width="200"/>
>>>>>>> remote

<<<<<<< LOCAL CELL DELETED >>>>>>>
<img src="images/dorict.png" alt="drawing" width="200"/>

## Spark setup

In [ ]:
import $ivy.`org.apache.spark::spark-sql:3.1.1`
import $ivy.`org.typelevel::cats-core:2.3.0`
import $ivy.`com.lihaoyi::sourcecode:0.2.6`

import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.{functions => f}

> There is no problem in combining conventional Spark column expressions and doric columns. However, to avoid name clashes, we will use the prefix `f` for the former ones.

In [ ]:
val spark = org.apache.spark.sql.SparkSession.builder().appName("test").master("local").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

In [ ]:
import spark.implicits._

## Doric setup

Adding sonatype snapshot repository - proper versions coming soon!

In [ ]:
import coursierapi.MavenRepository

interp.repositories.update(
  interp.repositories() ::: List(MavenRepository.of("https://oss.sonatype.org/content/repositories/snapshots"))
)

In [ ]:
import $ivy.`org.hablapps::doric:0.0.0+83-2a1f5312-SNAPSHOT`

Specific _doric_ imports:

In [ ]:
import habla.doric._
import habla.doric.functions._

## Some compelling reasons for using `doric`

Here it's a list of use cases for doric: 
* Case 1: Get rid of malformed column expressions at compile time 
* Case 2: Avoid implicit type castings
* Case 3: Run DataFrames only when it is safe to do so
* Case 4: Get all errors at once
* Case 5: Modularize your business logic 

Let's start!

<<<<<<< REMOTE CELL DELETED >>>>>>>
Here it's a list of use cases for doric: 
* Case 1: Get rid of malformed column expressions at compile time 
* Case 2: Avoid implicit type castings
* Case 3: Run DataFrames only when it is safe to do so
* Case 4: Get all errors at once
* Case 5: Modularize your business logic 

As we will see shortly, you will achieve all these goodies: 
<<<<<<< local
* Without resorting to Datasets and sacrifying performance, i.e. sticking to DataFrames
* With minimal change in your code with respect conventional column expressions
<<<<<<< local
<<<<<<< local
=======
* Without resorting to Datasets and sacrificing performance, i.e. sticking to DataFrames
* With minimal change in your code with respect to conventional column expressions
>>>>>>> remote
* Without fully committing to a strong static typing discipline throughout your code
=======
* Without fully committing to a strong static typing discipline in all your code
>>>>>>> remote
=======
* Without fully committing to a strong static typing discipline throughout your code
>>>>>>> remote

Let's start!

## Case 1: Get rid of malformed column expressions at compile time

We can't mix apples and oranges, and Spark knows that. For instance, Spark complains if we try to add integers with booleans:

In [ ]:
val df = List(1,2,3).toDF.select($"value" * f.lit(true))

But it complains too late, with an exception raised at runtime. If we delay the creation of the DataFrame, the error dissapears ...  

In [ ]:
def df = List(1,2,3).toDF.select($"value" * f.lit(true))

... momentarily, until we eventually invoke that code:

In [ ]:
df

Using doric, there is no need to wait so long: errors will be reported at compile-time!

In [ ]:
// This doesn't compile
def df = List(1,2,3).toDF.select(col[Int]("value") * lit(true))

Changes in column expressions are minimal: just annotate column references with the intended type, i.e. `col[Int]("value")`, instead of a plain `col("value")`. If you are not used to generic parameters, aliases `colInt`, `colString`, etc., are also available. We will use these aliases in the sequel.

> Naturally, this only works if you, the programmer, know the intended type of the column at compile-time. In a pure dynamic setting, doric is useless. Note, however, that you don't need to know in advance the whole row type as with `Dataset`s. In this way, doric sits between a whole-hearted static setting and a purely dynamic one. It offers type-safety at a minimum cost, without compromising performance.

We can also use doric columns within the context of a `withColumn` expression, or, in general, wherever we use plain columns: `join`, `filter`, etc.:

In [ ]:
List(1,2,3).toDF.withColumn("other", colInt("value") * lit(1))
List(1,2,3).toDF.filter(colInt("value") > lit(3))

Join expressions are explained in a separate [notebook](joins.ipynb) in more detail.

## Case 2: Explicitly avoid implicit type castings

Implicit type conversions in Spark are pervasive. For instance, the following code won't cause Spark to complain at all:

In [ ]:
val df0 = spark.range(1,10).withColumn("x", f.concat(f.col("id"), f.lit("jander"))) 

which means that an implicit conversion from integer to string is in effect:

In [ ]:
df0.select(f.col("x")).show

Assuming that you are certain that your column holds vales of type `bigint`, the same code in doric won't compile:

In [ ]:
val df = spark.range(1,10).toDF.withColumn("x", concat(colLong("id"), "jander".lit))

> Note that the Spark type `bigint` corresponds to the Scala type `Long`. The correspondences between Spark and Scala types in doric is the same as the one established in `Dataset`s by `Encoder` instances.

Still, doric will allow you to perform that operation provided that you explicitly enact the conversion:

In [ ]:
val df = spark.range(1,10).toDF.withColumn("x", concat(colLong("id").cast[String], "jander".lit))
df.show

Let's also consider the following example:

In [ ]:
val dfEq = List((1, "1"), (1, " 1"), (1, " 1 ")).toDF("int", "str")
dfEq.withColumn("eq", f.col("int") === f.col("str"))

What would you expect to be the result? Well, it all depends on the implicit conversion that Spark chooses to apply, if at all: 1) it may return false for the new column, given that the types of both input columns differ, thus choosing to apply no conversion; 2) it may convert the integer column into a string column; 3) it may convert strings to integers. Let's see what happens:

In [ ]:
dfEq.show

Option 3 wins, but you can only learn this by trial and error. With doric, you can depart from all this magic and explicitly cast types, if you desired so:

In [ ]:
// Option 1, no castings: compile error

In [ ]:
dfEq.withColumn("eq", colInt("int") === colString("str")).show

In [ ]:
// Option 2, casting from int to string

In [ ]:
dfEq.withColumn("eq", colInt("int").cast[String] === colString("str")).show

In [ ]:
// Option 3, casting from string to int, not safe!

In [ ]:
dfEq.withColumn("eq", colInt("int") === colString("str").unsafeCast[Int]).show

Note that we can't simply `cast` an string to an integer, since this conversion is partial. If the programmer insists in doing this unsafe casting, doric will force her to explicitly acknowledge this fact using the conversion function `unsafeCast`.

> It's all about being explicit in order to enhance readability and avoid unexpected behaviours at runtime. Doric is a coding accelerator!

## Case 3: Don't let your DataFrame run if it shouldn't

Let's suppose that your DataFrame contains a reference to a non-existing column. No problem, Spark will detect that and will complain with an exception at runtime:

In [ ]:
List(1,2,3).toDF.select(f.col("id")+1)

Now, let's assume that the column exists but its type is not what we expected. Spark won't be able to detect that, since type expectations are not encoded in plain columns. Thus, the following code will compile and execute without errors:

In [ ]:
val df = List("1","2","three").toDF.select(f.col("value") + 1)

and we will be able to run the DataFrame:

In [ ]:
df.show

obtaining `null` values and garbage results, in general.

Using doric we can prevent the creation of the DataFrame, since column expressions are typed: 

In [ ]:
val df = List("1","2","three").toDF.select(colInt("value") + 1.lit)

If the column doesn't exist, it will complain with a similar message to that given by Spark:

In [ ]:
val df = List("1","2","three").toDF.select(colInt("id") + 1.lit)

But note that the location of the error is also included. This will prove immensely useful, as we will see later on!

## Case 4: Get all errors at once!

Given the following DataFrame: 

In [ ]:
val dfadd = List((1,2),(3,4)).toDF("int1", "int2")

let's try to add both columns as follows:

In [ ]:
dfadd.withColumn("add", f.col("Int_1") + f.col("Int_2"))

Rightly, Spark complains because column "Int_1" doesn't exist. Let's fix that problem:

In [ ]:
dfadd.withColumn("add", f.col("int1") + f.col("Int_2"))

Ooops, another error. Fortunately, this is the last one:

In [ ]:
dfadd.withColumn("add", f.col("int1") + f.col("int2"))

But, why didn't Spark give us all errors at once? Well, a plain fail-fast strategy for error reporting is simpler. Unlike Spark, doric won't stop at the first error, and will keep accumulating all errors until no further one is found:

In [ ]:
dfadd.withColumn("add", colInt("int_1") + colInt("int_2"))

## Case 5: Modularity FTW!

Let's pretend that our business logic is very complex, and modularised in different functions. For instance:

In [ ]:
val col1: Column = f.col("int_1")
val col2: Column = f.col("int2")
val addColumns: Column = col1 + col2

There is an error when referring to the first column and Spark reports it:

In [ ]:
dfadd.withColumn("add", addColumns)

But, Spark does not give a clue about the exact source of the error. It marks the error in the `withColumn` method, but the actual problem is elsewhere, in expression `col1`. You have no choice but diving into the code and perform a brute exhaustive search. 

Using doric, we can modularise our code without remorse:

In [ ]:
val col1: DoricColumn[Int] = colInt("int_1")
val col2: DoricColumn[Int]  = colString("int2").unsafeCast[Int]
val addColumns: DoricColumn[Int] = col1 + col2

When we attempt to compose the DataFrame: 

In [ ]:
dfadd.withColumn("add", addColumns)

we will get not only the errors, but the exact location of the culprit: 

```
habla.doric.DoricMultiError: Found 2 errors in withColumn
	Cannot resolve column name "int_1" among (int1, int2)
		located at . (cmd83.sc:1)
	The column with name 'int2' is of type IntegerType and it was expected to be StringType
		located at . (cmd83.sc:2)
```

<<<<<<< local
<<<<<<< local
As you can see, errors are reported referring to the source files (`cmd83.sc`) and line numbers (`1` and `2`, respectively) where they are located. If you are using an IDE, you will additionally obtain an hyperlink to the error. Isn't that nice? :)
=======
As you can see, errors are reported referring to the source files (`cmd83.sc`) and line numbers (`1` and `2`, respectively) where they are located. If you are using an IDE, you will additionally obtain an hyperlink to the error.
>>>>>>> remote
=======
As you can see, errors are reported referring to the source files (`cmd83.sc`) and line numbers (`1` and `2`, respectively) where they are located. If you are using an IDE, you will additionally obtain an hyperlink to the error. Isn't that nice? :)
>>>>>>> remote